In [40]:
import pandas as pd
df = pd.read_csv('./data/movies_train.csv')
testdf = pd.read_csv('./data/movies_test.csv')
# 지수 표현 제거
pd.options.display.float_format = '{:.5f}'.format

In [2]:
df.corr()

,time,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num
time,1.00000,0.26468,0.30673,0.62321,0.11415,0.44145
dir_prev_bfnum,0.26468,1.00000,0.13182,0.32352,0.08382,0.28318
dir_prev_num,0.30673,0.13182,1.00000,0.45071,0.01401,0.25967
num_staff,0.62321,0.32352,0.45071,1.00000,0.07787,0.54426
num_actor,0.11415,0.08382,0.01401,0.07787,1.00000,0.11118
box_off_num,0.44145,0.28318,0.25967,0.54426,0.11118,1.00000


# 전처리

In [41]:
# column 생성
def make_col(dataframe, column_name, mapper) :
    new = []
    for i in dataframe[column_name] :
        if i in mapper.keys() :
            new.append(mapper.get(i))
        else :
            new.append(1)
    dataframe['new_' + str(column_name)] = pd.DataFrame(new)
    del dataframe[column_name]
    return dataframe

## title
title 은 제거

In [42]:
del df['title']
del testdf['title']

## distributor

In [43]:
a = df.groupby(['distributor']).mean()['box_off_num'].sort_values(ascending=False)

distributor_mapper = {}
for i in range(len(a)) :
    if a[i] >= 1000000 :
        distributor_mapper.update({a.index[i] : 5 })
    elif a[i] >= 100000 :
        distributor_mapper.update({a.index[i] : 4 })
    elif a[i] >= 10000 :
        distributor_mapper.update({a.index[i] : 3 })
    elif a[i] >= 1000 :
        distributor_mapper.update({a.index[i] : 2 })
    else :
        distributor_mapper.update({a.index[i] : 1 })

df = make_col(df, 'distributor', distributor_mapper )
testdf = make_col(testdf, 'distributor', distributor_mapper )

### 배급사별 box_off_num 의 평균에 따른 head, tail

In [18]:
df.groupby(['distributor']).mean()['box_off_num'].sort_values(ascending=False).head(20)

distributor
CJ E&M Pictures    4.122337e+06
(주)쇼박스             3.386656e+06
(주)아이필름코퍼레이션       3.117859e+06
쇼박스(주)미디어플렉스       2.634823e+06
영구아트무비             2.541603e+06
CJ E&M 영화부문        2.505309e+06
(주)NEW             2.184405e+06
CJ 엔터테인먼트          2.162742e+06
이십세기폭스코리아(주)       1.637046e+06
SK텔레콤(주)           1.383867e+06
CGV아트하우스           1.318189e+06
롯데엔터테인먼트           1.238149e+06
전망좋은영화사            1.214237e+06
주머니필름              1.201944e+06
NEW                1.019017e+06
싸이더스               9.634390e+05
시네마서비스             8.968432e+05
보람엔터테인먼트           8.496350e+05
스튜디오 블루            7.716990e+05
씨네그루(주)키다리이엔티      7.551840e+05
Name: box_off_num, dtype: float64

In [19]:
df.groupby(['distributor']).mean()['box_off_num'].sort_values(ascending=False).tail(20)

distributor
웃기씨네           464.5
판다미디어          432.0
(주)더픽쳐스        429.0
이상우필름          339.0
(주)온비즈넷        310.0
(주)유비네트워크      303.0
(주)휘엔터테인먼트     228.0
크리에이티브컴즈(주)    188.0
ysfilm         172.0
에스와이코마드        163.0
건시네마           162.0
(주)씨엠닉스        136.0
위더스필름           90.0
인터콘미디어          79.0
한국YWCA연합회       71.0
나우콘텐츠           54.0
위드시네마           46.0
사람과 사람들         42.0
고구마공작소           8.0
인피니티엔터테인먼트       2.0
Name: box_off_num, dtype: float64

## genre

### genre 에 따른 box_off_num 평균 확인

In [83]:
df.groupby(['genre']).mean()['box_off_num'].sort_values(ascending=False).index

Index(['느와르', '액션', 'SF', '코미디', '드라마', '공포', '미스터리', '멜로/로맨스', '애니메이션',
       '서스펜스', '다큐멘터리', '뮤지컬'],
      dtype='object', name='genre')

In [84]:
df.groupby(['genre']).mean()['box_off_num'].sort_values(ascending=False)

genre
느와르       2.263695e+06
액션        2.203974e+06
SF        1.788346e+06
코미디       1.193914e+06
드라마       6.256898e+05
공포        5.908325e+05
미스터리      5.275482e+05
멜로/로맨스    4.259680e+05
애니메이션     1.819267e+05
서스펜스      8.261100e+04
다큐멘터리     6.717226e+04
뮤지컬       6.627000e+03
Name: box_off_num, dtype: float64

In [185]:
df['genre'].value_counts()

드라마       221
다큐멘터리      93
멜로/로맨스     78
코미디        53
공포         42
액션         28
느와르        27
애니메이션      21
미스터리       17
SF         13
뮤지컬         5
서스펜스        2
Name: genre, dtype: int64

### genre_version1

In [168]:
df = pd.get_dummies(df, columns=['genre'], prefix='genre')
testdf = pd.get_dummies(testdf, columns=['genre'], prefix='genre')

### genre_version2

version1, 3 보다 좋음

영화 종류 묶기 + box_off_num의 평균에 따라 비선호 영화 묶기

=> 1466134.4524178207	

* 드라마 멜로/로맨스6

* 느와르 액션5

* sf4

* 코미디3

* 공포 미스터리 서스펜스2

* 애니메이션 다큐멘터리 뮤지컬1

In [44]:
genre_mapper = {'느와르' : 5 , '액션' : 5, 'SF' : 4, '코미디' : 3, '드라마' : 6, '공포' : 2, '미스터리' : 2, '멜로/로맨스' : 6, '애니메이션' : 1,
       '서스펜스' : 2, '다큐멘터리' : 1, '뮤지컬' : 1} 

df = make_col(df, 'genre', genre_mapper )
testdf = make_col(testdf, 'genre', genre_mapper )

### genre_version3

box_off_num 평균에 따라 임의로 분류

=> 1467553.925063711 

In [ ]:
느와르       4
액션        4
SF        3
코미디       3
드라마       2
공포        2
미스터리      2
멜로/로맨스    2
애니메이션     1
서스펜스     1
다큐멘터리     1
뮤지컬       1

In [210]:
genre_mapper = {'느와르' : 4 , '액션' : 4, 'SF' : 3, '코미디' : 3, '드라마' : 2, '공포' : 2, '미스터리' : 2, '멜로/로맨스' : 2, '애니메이션' : 1,
       '서스펜스' : 1, '다큐멘터리' : 1, '뮤지컬' : 1} 

df = make_col(df, 'genre', genre_mapper )
testdf = make_col(testdf, 'genre', genre_mapper )

## release_time

* days 그대로 사용해보자

=> sklearn,  1484884.4136596955

=> 전처리 하는 것이 더 좋음


In [45]:
def release_time(df) :
    years, months, days = [], [], []
    for line in df['release_time'] :
        a = line.split('-')
        years.append(int(a[0]))
        months.append(int(a[1]))
        days.append(int(a[2]))
        
    new_days = []
    for i in range(len(days)) :
        if int(days[i]) >= 20 :
            new_days.append(3)
        elif int(days[i]) >= 10 :
            new_days.append(2)
        else :
            new_days.append(1)    
            
    df['release_year'] = pd.DataFrame(years)
    df['release_month'] = pd.DataFrame(months)
    df['release_day'] = pd.DataFrame(new_days)

    del df['release_time']
    return df

df = release_time(df)
testdf = release_time(testdf)

## time

전처리 이전과 비교하였을 때, box_off_num 과의 상관관계는 낮아졌다 ( 0.44 -> 0.29 ) 

하지만, 전처리 이후 성능이 개선됨

=> 1472434.761373017 ( 130, 70, ~ )

=> 1472973.0197688688 ( 130, 100, 70, ~ )

In [46]:
def time(df) :
    new_time = []
    for i in df['time'] :
        if i >= 130 :
            new_time.append(3)
        elif i > 70 : 
            new_time.append(2)
        else :
            new_time.append(1)

    df['new_time'] = pd.DataFrame(new_time)
    del df['time']
    return df

df = time(df)
testdf = time(testdf)

## screening_rat

### mapping_version1

version1 이  version2보다 더 좋음

In [88]:
screening_rat_mapper = {'15세 관람가' : 3, '12세 관람가' : 2, '전체 관람가' : 2, '청소년 관람불가' : 1} 

df = make_col(df, 'screening_rat', screening_rat_mapper )
testdf = make_col(testdf, 'screening_rat', screening_rat_mapper )

### mapping_version2

In [83]:
def screen_rat(df) :
    tmp = sorted(df['screening_rat'].unique(), reverse=False)

    mapping = {}
    num = 1
    for i in tmp :
        mapping.update({i : num})
        num += 1

    return make_col(df, 'screening_rat', mapping)

In [84]:
df = screen_rat(df)
testdf = screen_rat(testdf)

### mapping_version3

version1과 큰 차이는 나지 않지만 더 좋음

=> 1483501.7131900766	

In [47]:
df = pd.get_dummies(df, columns=['screening_rat'], prefix='screening_rat')
testdf = pd.get_dummies(testdf, columns=['screening_rat'], prefix='screening_rat')

### 관람 연령 제한에 따른 box_off_num 평균, 영화 수 

In [51]:
df.groupby(['screening_rat']).mean()['box_off_num'].sort_values(ascending=False)

screening_rat
15세 관람가     1.247519e+06
12세 관람가     8.449809e+05
청소년 관람불가    3.641813e+05
전체 관람가      1.351005e+05
Name: box_off_num, dtype: float64

In [52]:
df.groupby(['screening_rat']).count()['box_off_num'].sort_values(ascending=False)

screening_rat
청소년 관람불가    204
15세 관람가     202
12세 관람가     102
전체 관람가       92
Name: box_off_num, dtype: int64

## director

In [48]:
del df['director']
del testdf['director']

## dir_prev_bfnum

### version1
* 결측치 0으로 채워 넣음

In [35]:
df = df.fillna(0)
testdf = testdf.fillna(0)

### version2
* 결측치가 많은 column이므로 제거

=> 1483572.7123881471	

In [11]:
del df['dir_prev_bfnum']
del testdf['dir_prev_bfnum']

### version3
* log scale

In [49]:
df = df.fillna(0)
testdf = testdf.fillna(0)
df['dir_prev_bfnum'] = np.log1p(df['dir_prev_bfnum'])
testdf['dir_prev_bfnum'] = np.log1p(testdf['dir_prev_bfnum'])

## dir_prev_num

* column제거하였더니 점수 떨어짐

=> 1471931.2287102737	

### version1

In [ ]:
del df['dir_prev_num']
del testdf['dir_prev_num']

### version2

In [50]:
df['dir_prev_num'] = np.log1p(df['dir_prev_num'])
testdf['dir_prev_num'] = np.log1p(testdf['dir_prev_num'])

## num_staff

staff 는 범주화하지 않는 것이 더 좋다!

=>	1536288.9882051074

### staff_version1

In [68]:
def staff(dataframe) : 
    new_staff = []
    for i in dataframe['num_staff'] :
        if i == 0 :
            new_staff.append(0)
        elif i <= 100 :
            new_staff.append(1)
        elif i <= 300 :
            new_staff.append(2)        
        elif i <= 500 :
            new_staff.append(3)
        else :
            new_staff.append(4)
    dataframe['new_staff'] = pd.DataFrame(new_staff)
    del dataframe['num_staff']
    
    return dataframe

In [69]:
df = staff(df)
testdf = staff(testdf)

In [70]:
df.groupby(['new_staff']).mean()['box_off_num'].sort_values(ascending=False)

new_staff
4    4.132596e+06
3    1.980997e+06
2    6.495566e+05
1    1.164862e+05
0    7.805562e+03
Name: box_off_num, dtype: float64

### staff_version2
* log scale

In [51]:
df['num_staff'] = np.log1p(df['num_staff'])
testdf['num_staff'] = np.log1p(testdf['num_staff'])

## num_actor

column 제거했더니 ?

=> sklearn, 1470532.643567296	

### version1

In [13]:
del df['num_actor']
del testdf['num_actor']

### version2

In [52]:
df['num_actor'] = np.log1p(df['num_actor'])
testdf['num_actor'] = np.log1p(testdf['num_actor'])

# 상관관계

In [53]:
df.corr().sort_values(by='box_off_num', ascending = False)

,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,new_distributor,new_genre,release_year,release_month,release_day,new_time,screening_rat_12세 관람가,screening_rat_15세 관람가,screening_rat_전체 관람가,screening_rat_청소년 관람불가
box_off_num,0.28229,0.25906,0.37161,0.12675,1.00000,0.45016,0.09595,-0.00250,0.01910,-0.01102,0.29498,0.03390,0.21037,-0.13353,-0.13518
new_distributor,0.42614,0.40410,0.64305,0.11406,0.45016,1.00000,0.08755,-0.05986,-0.02799,-0.02770,0.25603,0.03496,0.25791,-0.16415,-0.16016
num_staff,0.42734,0.40361,1.00000,0.16218,0.37161,0.64305,0.36634,-0.07375,-0.06090,-0.00971,0.26523,0.00926,0.29240,-0.38197,-0.00851
new_time,0.21015,0.17047,0.26523,0.04930,0.29498,0.25603,0.10255,-0.05254,0.00813,0.02927,1.00000,-0.03082,0.04755,-0.07623,0.03498
dir_prev_bfnum,1.00000,0.91628,0.42734,0.04462,0.28229,0.42614,0.14997,0.08791,0.02161,0.02273,0.21015,0.00792,0.18106,-0.17775,-0.05171
dir_prev_num,0.91628,1.00000,0.40361,0.04234,0.25906,0.40410,0.14210,0.11845,0.03299,0.03021,0.17047,0.00580,0.19858,-0.17439,-0.07006
screening_rat_15세 관람가,0.18106,0.19858,0.29240,0.07762,0.21037,0.25791,0.06829,-0.07186,-0.05132,-0.01894,0.04755,-0.32242,1.00000,-0.30318,-0.51133
num_actor,0.04462,0.04234,0.16218,1.00000,0.12675,0.11406,0.03074,-0.09980,0.00099,0.00401,0.04930,-0.03625,0.07762,-0.04977,-0.01084
new_genre,0.14997,0.14210,0.36634,0.03074,0.09595,0.08755,1.00000,-0.01883,-0.02042,0.00124,0.10255,0.01585,0.06829,-0.43499,0.25017
screening_rat_12세 관람가,0.00792,0.00580,0.00926,-0.03625,0.03390,0.03496,0.01585,-0.02142,0.04292,-0.02659,-0.03082,1.00000,-0.32242,-0.19260,-0.32483


# 모델링

In [57]:
import numpy as np
from sklearn.metrics import make_scorer

def rmse(predict, actual):
    predict = np.array(predict)
    actual = np.array(actual)

    distance = predict - actual
    square_distance = distance ** 2
    mean_square_distance = square_distance.mean()
    score = np.sqrt(mean_square_distance)
    
    return score

rmse_score = make_scorer(rmse)
rmse_score

make_scorer(rmse)

## sklearn_Linear Regression
* dir_prev_num, num_staff, num_actor log scale : 1655506.2449801604

In [158]:
train_Y = df['box_off_num']
train_X = df.drop(['box_off_num'], axis=1)

test_X = testdf

In [159]:
from sklearn import linear_model

lm = linear_model.LinearRegression()
lm.fit(train_X, train_Y)

prediction = lm.predict(test_X)

## sklearn_RandomForestRegressor

=> 1528294.5021972428	 ( epoch X )

In [240]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

model = RandomForestRegressor(n_estimators=300,
                              n_jobs=-1,
                              random_state=46)
#score = cross_val_score(model, X_train, y_train, cv=20, scoring=rmse_score).mean()

In [242]:
train_Y = df['box_off_num']
train_X = df.drop(['box_off_num'], axis=1)

test_X = testdf

model.fit(train_X, train_Y)


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=300, n_jobs=-1, oob_score=False,
                      random_state=46, verbose=0, warm_start=False)

In [245]:
prediction = model.predict(test_X)

array([1.37224077e+06, 1.37712202e+06, 2.91496949e+06, 2.35731362e+06,
       2.18836183e+06, 3.83021800e+04, 3.05253000e+03, 1.25348333e+03,
       4.89507000e+03, 2.70432873e+05, 4.52084997e+05, 1.09368079e+06,
       2.43055558e+06, 3.80952000e+03, 1.54969315e+06, 2.49022232e+06,
       4.11237000e+03, 2.80051673e+06, 9.91018117e+05, 4.01313674e+06,
       3.31340000e+02, 2.75170133e+04, 1.04387323e+05, 2.15437233e+04,
       2.86300000e+02, 3.14427729e+06, 2.01759000e+03, 2.63354868e+06,
       9.63927000e+03, 7.76519000e+03, 4.01482357e+06, 2.53251757e+05,
       3.11146333e+03, 5.57792827e+05, 6.60871667e+03, 2.34236933e+04,
       1.94996333e+03, 4.38108000e+03, 4.30301000e+03, 3.68430000e+02,
       5.67607333e+03, 2.82403647e+05, 9.73643447e+05, 3.93951430e+05,
       1.05306354e+06, 1.88549011e+06, 8.36590000e+02, 1.48209400e+04,
       7.02168903e+05, 7.65833333e+02, 6.29552667e+03, 1.05540849e+06,
       5.76044333e+03, 1.29539582e+06, 1.30844980e+06, 2.60748967e+04,
      

## LGBM

=> 1480188.794477261

In [62]:
df.columns = ['dir_prev_bfnum', 'dir_prev_num', 'num_staff', 'num_actor',
       'box_off_num', 'new_distributor', 'new_genre', 'release_year',
       'release_month', 'release_day', 'new_time', 'screening_rat_12',
       'screening_rat_15', 'screening_rat_all',
       'screening_rat_19']
testdf.columns = ['dir_prev_bfnum', 'dir_prev_num', 'num_staff', 'num_actor',
       'new_distributor', 'new_genre', 'release_year',
       'release_month', 'release_day', 'new_time', 'screening_rat_12',
       'screening_rat_15', 'screening_rat_all',
       'screening_rat_19']

In [63]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
k_fold = KFold(n_splits=19, shuffle=True, random_state = 777)

model = lgb.LGBMRegressor(random_state = 777, n_estimators=10000)
X_train, X_test, Y_train = df.drop(['box_off_num'], axis=1), testdf, df['box_off_num']


params = {
    'learning_rate' : [0.001, 0.0001, 0.00001],
    'min_child_samples' : [20, 30]}

gs = GridSearchCV(estimator=model,
                 param_grid = params,
                 scoring=rmse_score,
                 cv=k_fold)

In [64]:
gs.fit(X_train, Y_train)

GridSearchCV(cv=KFold(n_splits=19, random_state=777, shuffle=True),
             error_score=nan,
             estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None,
                                     colsample_bytree=1.0,
                                     importance_type='split', learning_rate=0.1,
                                     max_depth=-1, min_child_samples=20,
                                     min_child_weight=0.001, min_split_gain=0.0,
                                     n_estimators=10000, n_jobs=-1,
                                     num_leaves=31, objective=None,
                                     random_state=777, reg_alpha=0.0,
                                     reg_lambda=0.0, silent=True, subsample=1.0,
                                     subsample_for_bin=200000,
                                     subsample_freq=0),
             iid='deprecated', n_jobs=None,
             param_grid={'learning_rate': [0.001, 0.0001, 1e-05],
              

In [65]:
gs.best_params_

{'learning_rate': 1e-05, 'min_child_samples': 30}

In [66]:
model = lgb.LGBMRegressor(random_state = 777, n_estimators=10000, learning_rate= 1e-05, min_child_samples= 30)
models = []
for train_idx, val_idx in k_fold.split(X_train) :
    x_t = X_train.iloc[train_idx]
    y_t = Y_train.iloc[train_idx]
    x_val = X_train.iloc[val_idx]
    y_val = Y_train.iloc[val_idx]
    
    models.append(model.fit(x_t, y_t, eval_set = (x_val, y_val), early_stopping_rounds=1000, verbose=100))
    

Training until validation scores don't improve for 1000 rounds
[100]	valid_0's l2: 7.88774e+12
[200]	valid_0's l2: 7.88605e+12
[300]	valid_0's l2: 7.88436e+12
[400]	valid_0's l2: 7.88267e+12
[500]	valid_0's l2: 7.88098e+12
[600]	valid_0's l2: 7.8793e+12
[700]	valid_0's l2: 7.87762e+12
[800]	valid_0's l2: 7.87594e+12
[900]	valid_0's l2: 7.87427e+12
[1000]	valid_0's l2: 7.87259e+12
[1100]	valid_0's l2: 7.87092e+12
[1200]	valid_0's l2: 7.86926e+12
[1300]	valid_0's l2: 7.86759e+12
[1400]	valid_0's l2: 7.86593e+12
[1500]	valid_0's l2: 7.86427e+12
[1600]	valid_0's l2: 7.86261e+12
[1700]	valid_0's l2: 7.86096e+12
[1800]	valid_0's l2: 7.85931e+12
[1900]	valid_0's l2: 7.85766e+12
[2000]	valid_0's l2: 7.85601e+12
[2100]	valid_0's l2: 7.85437e+12
[2200]	valid_0's l2: 7.85273e+12
[2300]	valid_0's l2: 7.85109e+12
[2400]	valid_0's l2: 7.84945e+12
[2500]	valid_0's l2: 7.84782e+12
[2600]	valid_0's l2: 7.84619e+12
[2700]	valid_0's l2: 7.84456e+12
[2800]	valid_0's l2: 7.84294e+12
[2900]	valid_0's l2: 7.

[4300]	valid_0's l2: 9.16512e+11
[4400]	valid_0's l2: 9.15883e+11
[4500]	valid_0's l2: 9.15259e+11
[4600]	valid_0's l2: 9.14651e+11
[4700]	valid_0's l2: 9.14047e+11
[4800]	valid_0's l2: 9.13445e+11
[4900]	valid_0's l2: 9.12846e+11
[5000]	valid_0's l2: 9.1225e+11
[5100]	valid_0's l2: 9.11659e+11
[5200]	valid_0's l2: 9.11072e+11
[5300]	valid_0's l2: 9.10489e+11
[5400]	valid_0's l2: 9.09908e+11
[5500]	valid_0's l2: 9.0933e+11
[5600]	valid_0's l2: 9.08755e+11
[5700]	valid_0's l2: 9.08184e+11
[5800]	valid_0's l2: 9.07615e+11
[5900]	valid_0's l2: 9.07048e+11
[6000]	valid_0's l2: 9.06481e+11
[6100]	valid_0's l2: 9.05915e+11
[6200]	valid_0's l2: 9.05353e+11
[6300]	valid_0's l2: 9.04794e+11
[6400]	valid_0's l2: 9.04237e+11
[6500]	valid_0's l2: 9.03683e+11
[6600]	valid_0's l2: 9.03132e+11
[6700]	valid_0's l2: 9.02584e+11
[6800]	valid_0's l2: 9.02038e+11
[6900]	valid_0's l2: 9.01496e+11
[7000]	valid_0's l2: 9.00956e+11
[7100]	valid_0's l2: 9.00418e+11
[7200]	valid_0's l2: 8.99884e+11
[7300]	valid

[8700]	valid_0's l2: 4.69054e+12
[8800]	valid_0's l2: 4.68894e+12
[8900]	valid_0's l2: 4.68733e+12
[9000]	valid_0's l2: 4.68573e+12
[9100]	valid_0's l2: 4.68414e+12
[9200]	valid_0's l2: 4.68254e+12
[9300]	valid_0's l2: 4.68095e+12
[9400]	valid_0's l2: 4.67936e+12
[9500]	valid_0's l2: 4.67777e+12
[9600]	valid_0's l2: 4.67619e+12
[9700]	valid_0's l2: 4.67461e+12
[9800]	valid_0's l2: 4.67303e+12
[9900]	valid_0's l2: 4.67145e+12
[10000]	valid_0's l2: 4.66976e+12
Did not meet early stopping. Best iteration is:
[10000]	valid_0's l2: 4.66976e+12
Training until validation scores don't improve for 1000 rounds
[100]	valid_0's l2: 1.16835e+13
[200]	valid_0's l2: 1.16809e+13
[300]	valid_0's l2: 1.16782e+13
[400]	valid_0's l2: 1.16756e+13
[500]	valid_0's l2: 1.1673e+13
[600]	valid_0's l2: 1.16704e+13
[700]	valid_0's l2: 1.16677e+13
[800]	valid_0's l2: 1.16651e+13
[900]	valid_0's l2: 1.16625e+13
[1000]	valid_0's l2: 1.16599e+13
[1100]	valid_0's l2: 1.16573e+13
[1200]	valid_0's l2: 1.16547e+13
[1300]

[2500]	valid_0's l2: 5.51672e+12
[2600]	valid_0's l2: 5.51438e+12
[2700]	valid_0's l2: 5.51205e+12
[2800]	valid_0's l2: 5.50972e+12
[2900]	valid_0's l2: 5.50739e+12
[3000]	valid_0's l2: 5.50506e+12
[3100]	valid_0's l2: 5.50274e+12
[3200]	valid_0's l2: 5.50042e+12
[3300]	valid_0's l2: 5.4981e+12
[3400]	valid_0's l2: 5.49579e+12
[3500]	valid_0's l2: 5.49348e+12
[3600]	valid_0's l2: 5.49118e+12
[3700]	valid_0's l2: 5.48887e+12
[3800]	valid_0's l2: 5.48657e+12
[3900]	valid_0's l2: 5.48428e+12
[4000]	valid_0's l2: 5.48199e+12
[4100]	valid_0's l2: 5.4797e+12
[4200]	valid_0's l2: 5.47741e+12
[4300]	valid_0's l2: 5.47513e+12
[4400]	valid_0's l2: 5.47285e+12
[4500]	valid_0's l2: 5.47058e+12
[4600]	valid_0's l2: 5.4683e+12
[4700]	valid_0's l2: 5.46604e+12
[4800]	valid_0's l2: 5.46377e+12
[4900]	valid_0's l2: 5.46151e+12
[5000]	valid_0's l2: 5.45925e+12
[5100]	valid_0's l2: 5.45699e+12
[5200]	valid_0's l2: 5.45474e+12
[5300]	valid_0's l2: 5.45249e+12
[5400]	valid_0's l2: 5.45025e+12
[5500]	valid_

[6800]	valid_0's l2: 5.1258e+12
[6900]	valid_0's l2: 5.12277e+12
[7000]	valid_0's l2: 5.11993e+12
[7100]	valid_0's l2: 5.11722e+12
[7200]	valid_0's l2: 5.11451e+12
[7300]	valid_0's l2: 5.11182e+12
[7400]	valid_0's l2: 5.10912e+12
[7500]	valid_0's l2: 5.10643e+12
[7600]	valid_0's l2: 5.10371e+12
[7700]	valid_0's l2: 5.101e+12
[7800]	valid_0's l2: 5.09829e+12
[7900]	valid_0's l2: 5.09559e+12
[8000]	valid_0's l2: 5.09289e+12
[8100]	valid_0's l2: 5.09019e+12
[8200]	valid_0's l2: 5.0875e+12
[8300]	valid_0's l2: 5.08482e+12
[8400]	valid_0's l2: 5.08214e+12
[8500]	valid_0's l2: 5.07946e+12
[8600]	valid_0's l2: 5.07679e+12
[8700]	valid_0's l2: 5.07412e+12
[8800]	valid_0's l2: 5.07147e+12
[8900]	valid_0's l2: 5.06881e+12
[9000]	valid_0's l2: 5.06615e+12
[9100]	valid_0's l2: 5.06351e+12
[9200]	valid_0's l2: 5.06086e+12
[9300]	valid_0's l2: 5.05823e+12
[9400]	valid_0's l2: 5.05559e+12
[9500]	valid_0's l2: 5.05296e+12
[9600]	valid_0's l2: 5.05034e+12
[9700]	valid_0's l2: 5.04771e+12
[9800]	valid_0

[600]	valid_0's l2: 1.44163e+12
[700]	valid_0's l2: 1.44013e+12
[800]	valid_0's l2: 1.43863e+12
[900]	valid_0's l2: 1.43714e+12
[1000]	valid_0's l2: 1.43565e+12
[1100]	valid_0's l2: 1.43417e+12
[1200]	valid_0's l2: 1.43269e+12
[1300]	valid_0's l2: 1.43121e+12
[1400]	valid_0's l2: 1.42974e+12
[1500]	valid_0's l2: 1.42828e+12
[1600]	valid_0's l2: 1.42681e+12
[1700]	valid_0's l2: 1.42536e+12
[1800]	valid_0's l2: 1.4239e+12
[1900]	valid_0's l2: 1.42246e+12
[2000]	valid_0's l2: 1.42101e+12
[2100]	valid_0's l2: 1.41957e+12
[2200]	valid_0's l2: 1.41814e+12
[2300]	valid_0's l2: 1.41671e+12
[2400]	valid_0's l2: 1.41528e+12
[2500]	valid_0's l2: 1.41386e+12
[2600]	valid_0's l2: 1.41244e+12
[2700]	valid_0's l2: 1.41103e+12
[2800]	valid_0's l2: 1.40962e+12
[2900]	valid_0's l2: 1.40822e+12
[3000]	valid_0's l2: 1.40682e+12
[3100]	valid_0's l2: 1.40543e+12
[3200]	valid_0's l2: 1.40403e+12
[3300]	valid_0's l2: 1.40265e+12
[3400]	valid_0's l2: 1.40127e+12
[3500]	valid_0's l2: 1.39989e+12
[3600]	valid_0'

[4700]	valid_0's l2: 3.0769e+12
[4800]	valid_0's l2: 3.07467e+12
[4900]	valid_0's l2: 3.07245e+12
[5000]	valid_0's l2: 3.07024e+12
[5100]	valid_0's l2: 3.06802e+12
[5200]	valid_0's l2: 3.06582e+12
[5300]	valid_0's l2: 3.06361e+12
[5400]	valid_0's l2: 3.06141e+12
[5500]	valid_0's l2: 3.05922e+12
[5600]	valid_0's l2: 3.05703e+12
[5700]	valid_0's l2: 3.05485e+12
[5800]	valid_0's l2: 3.05267e+12
[5900]	valid_0's l2: 3.05049e+12
[6000]	valid_0's l2: 3.04832e+12
[6100]	valid_0's l2: 3.04615e+12
[6200]	valid_0's l2: 3.04399e+12
[6300]	valid_0's l2: 3.04183e+12
[6400]	valid_0's l2: 3.03967e+12
[6500]	valid_0's l2: 3.03752e+12
[6600]	valid_0's l2: 3.03538e+12
[6700]	valid_0's l2: 3.03324e+12
[6800]	valid_0's l2: 3.0311e+12
[6900]	valid_0's l2: 3.02897e+12
[7000]	valid_0's l2: 3.02684e+12
[7100]	valid_0's l2: 3.02472e+12
[7200]	valid_0's l2: 3.0226e+12
[7300]	valid_0's l2: 3.02048e+12
[7400]	valid_0's l2: 3.01837e+12
[7500]	valid_0's l2: 3.01626e+12
[7600]	valid_0's l2: 3.01416e+12
[7700]	valid_

[8800]	valid_0's l2: 4.72726e+11
[8900]	valid_0's l2: 4.71831e+11
[9000]	valid_0's l2: 4.7094e+11
[9100]	valid_0's l2: 4.70052e+11
[9200]	valid_0's l2: 4.69167e+11
[9300]	valid_0's l2: 4.68285e+11
[9400]	valid_0's l2: 4.67407e+11
[9500]	valid_0's l2: 4.66533e+11
[9600]	valid_0's l2: 4.65653e+11
[9700]	valid_0's l2: 4.64773e+11
[9800]	valid_0's l2: 4.63897e+11
[9900]	valid_0's l2: 4.63023e+11
[10000]	valid_0's l2: 4.62153e+11
Did not meet early stopping. Best iteration is:
[10000]	valid_0's l2: 4.62153e+11
Training until validation scores don't improve for 1000 rounds
[100]	valid_0's l2: 8.58123e+11
[200]	valid_0's l2: 8.57311e+11
[300]	valid_0's l2: 8.56503e+11
[400]	valid_0's l2: 8.55697e+11
[500]	valid_0's l2: 8.54893e+11
[600]	valid_0's l2: 8.54092e+11
[700]	valid_0's l2: 8.53294e+11
[800]	valid_0's l2: 8.52499e+11
[900]	valid_0's l2: 8.51706e+11
[1000]	valid_0's l2: 8.50915e+11
[1100]	valid_0's l2: 8.50127e+11
[1200]	valid_0's l2: 8.49342e+11
[1300]	valid_0's l2: 8.48559e+11
[1400]

In [67]:
preds = []
for model in models :
    preds.append(model.predict(X_test))
len(preds)

19

In [68]:
import numpy as np
prediction = np.mean(preds, axis=0)

In [69]:
prediction

array([699630.66612452, 664558.74937633, 752588.20073902, 752588.20073902,
       752588.20073902, 610882.35519496, 607533.16349341, 607190.05158412,
       607190.05158412, 649055.51622155, 633221.87247778, 699630.66612452,
       699630.66612452, 607393.26104116, 833461.78383054, 857039.31624019,
       607707.24451382, 833461.78383054, 664558.74937633, 733676.02304695,
       607190.05158412, 611915.37709133, 628834.21110819, 607190.05158412,
       607190.05158412, 857039.31624019, 607379.99867696, 833461.78383054,
       611188.178438  , 610882.35519496, 833461.78383054, 614087.03144021,
       607795.18854546, 649120.95570837, 607795.18854546, 608558.68652572,
       607379.99867696, 607795.18854546, 607749.6484581 , 607190.05158412,
       607190.05158412, 612736.23968606, 733676.02304695, 614061.30072314,
       733676.02304695, 752588.20073902, 607190.05158412, 610882.35519496,
       733676.02304695, 607190.05158412, 610403.97296326, 733676.02304695,
       607693.98214961, 7

## sklearn_RandomForestRegressor 
+ RMSE + Hyper Parameter Tuning

* cv = 20 : 1460717.170338258	
* cv = 35 : 1454429.7489887767	(genre_mapping)
* 전처리 변경 후  : 1485610.1551896033
* cv = 19 : 1490913.1161172627	
* cv = 19 : 1455667.9935786778 (genre_get_dummies)
* cv = 35 : 1469407.8483563093	(genre_get_dummies)
* cv = 40 :  ????  (genre_mapping)

In [16]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=300,
                              n_jobs=-1,
                              random_state=37)

X_train, X_test, Y_train = df.drop(['box_off_num'], axis=1), testdf, df['box_off_num']

In [17]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

n_estimators = 300

# 랜덤 서치 횟수
num_epoch = 100

# hyperparameter 탐색 결과가 저장될 리스트
coarse_hyperparameters_list = []

for epoch in range(num_epoch):
    # max depth의 값을 2~100으로 설정
    max_depth = np.random.randint(low=2, high=100)

    # 적용할 feature의 비율을 0.1~1.0로 설정
    max_features = np.random.uniform(low=0.1, high=1.0)

    model = RandomForestRegressor(n_estimators=n_estimators,
                                  max_depth=max_depth,
                                  max_features=max_features,
                                  n_jobs=-1,
                                  random_state=37)

    # cross_val_score를 실행하여 rmse의 평균 계산
    # cv는 데이터 조각 개수
    score = cross_val_score(model, X_train, Y_train, cv=19, scoring=rmse_score).mean() 
    
    # hyperparameter 탐색 결과를 저장할 딕셔너리
    hyperparameters = {
        'epoch': epoch,
        'score': score,
        'n_estimators': n_estimators,
        'max_depth': max_depth,
        'max_features': max_features,
        
    }

    # hyperparameter 탐색 결과 저장
    coarse_hyperparameters_list.append(hyperparameters)

    # hyperparameter 탐색 결과를 출력
    print(f"{epoch:2} n_estimators = {n_estimators}, max_depth = {max_depth:2}, max_features = {max_features:.6f}, Score = {score:.5f}")

# coarse_hyperparameters_list를 DataFrame으로 변환
coarse_hyperparameters_list = pd.DataFrame.from_dict(coarse_hyperparameters_list)

# coarse_hyperparameters_list를 score가 낮은 순으로 정렬
coarse_hyperparameters_list = coarse_hyperparameters_list.sort_values(by="score")

print(coarse_hyperparameters_list.shape)

# coarse_hyperparameters_list의 상위 10개 출력
coarse_hyperparameters_list.head(10)

 0 n_estimators = 300, max_depth = 20, max_features = 0.142193, Score = 1345468.26286
 1 n_estimators = 300, max_depth =  8, max_features = 0.514011, Score = 1384029.54322
 2 n_estimators = 300, max_depth = 47, max_features = 0.932290, Score = 1427875.30536
 3 n_estimators = 300, max_depth = 81, max_features = 0.432295, Score = 1390614.60349
 4 n_estimators = 300, max_depth = 69, max_features = 0.531131, Score = 1389334.55699
 5 n_estimators = 300, max_depth = 88, max_features = 0.674587, Score = 1410988.37487
 6 n_estimators = 300, max_depth = 13, max_features = 0.542204, Score = 1385069.56721
 7 n_estimators = 300, max_depth = 77, max_features = 0.746038, Score = 1411864.02073
 8 n_estimators = 300, max_depth = 50, max_features = 0.505160, Score = 1389334.55699
 9 n_estimators = 300, max_depth = 21, max_features = 0.741678, Score = 1412884.50019
10 n_estimators = 300, max_depth = 68, max_features = 0.746141, Score = 1411864.02073
11 n_estimators = 300, max_depth = 78, max_features = 

96 n_estimators = 300, max_depth = 83, max_features = 0.841970, Score = 1427401.08585
97 n_estimators = 300, max_depth = 82, max_features = 0.333402, Score = 1367510.36325
98 n_estimators = 300, max_depth =  3, max_features = 0.436041, Score = 1315473.22835
99 n_estimators = 300, max_depth = 63, max_features = 0.730660, Score = 1411864.02073
(100, 5)


,epoch,score,n_estimators,max_depth,max_features
98,98,1315473.22835,300,3,0.43604
74,74,1323758.86591,300,3,0.53683
56,56,1326381.46558,300,2,0.54627
54,54,1328846.34013,300,5,0.38365
81,81,1338453.47394,300,3,0.20906
87,87,1343836.72691,300,79,0.10618
0,0,1345468.26286,300,20,0.14219
42,42,1346319.74140,300,37,0.25762
57,57,1346319.74140,300,97,0.23164
92,92,1346319.74140,300,84,0.24693


In [18]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

n_estimators = 300
num_epoch = 100
finer_hyperparameters_list = []

for epoch in range(num_epoch):
    # coarse 단계에서 찾은 뎁스와 피쳐비율 구간을 적용
    max_depth = np.random.randint(low=5, high=35)
    max_features = np.random.uniform(low=0.1, high=0.3)
    
    model = RandomForestRegressor(n_estimators=n_estimators,
                                  max_depth=max_depth,
                                  max_features=max_features,
                                  n_jobs=-1,
                                  random_state=37)

    score = cross_val_score(model, X_train, Y_train, cv=19, scoring=rmse_score).mean()
    hyperparameters = {
        'epoch': epoch,
        'score': score,
        'n_estimators': n_estimators,
        'max_depth': max_depth,
        'max_features': max_features,
    }

    finer_hyperparameters_list.append(hyperparameters)
    print(f"{epoch:2} n_estimators = {n_estimators}, max_depth = {max_depth:2}, max_features = {max_features:.6f}, Score = {score:.5f}")

finer_hyperparameters_list = pd.DataFrame.from_dict(finer_hyperparameters_list)
finer_hyperparameters_list = finer_hyperparameters_list.sort_values(by="score")
print(finer_hyperparameters_list.shape)

finer_hyperparameters_list.head(10)

 0 n_estimators = 300, max_depth = 18, max_features = 0.123752, Score = 1343826.82958
 1 n_estimators = 300, max_depth = 20, max_features = 0.234689, Score = 1353111.09913
 2 n_estimators = 300, max_depth = 19, max_features = 0.111767, Score = 1351791.72276
 3 n_estimators = 300, max_depth = 23, max_features = 0.177865, Score = 1344626.71751
 4 n_estimators = 300, max_depth = 19, max_features = 0.127120, Score = 1351791.72276
 5 n_estimators = 300, max_depth = 33, max_features = 0.224683, Score = 1355611.45489
 6 n_estimators = 300, max_depth = 19, max_features = 0.193755, Score = 1356717.57267
 7 n_estimators = 300, max_depth = 30, max_features = 0.120831, Score = 1343787.67248
 8 n_estimators = 300, max_depth = 18, max_features = 0.166764, Score = 1341578.10647
 9 n_estimators = 300, max_depth = 32, max_features = 0.270163, Score = 1346319.74140
10 n_estimators = 300, max_depth = 27, max_features = 0.271741, Score = 1345612.33417
11 n_estimators = 300, max_depth = 17, max_features = 

96 n_estimators = 300, max_depth = 21, max_features = 0.182149, Score = 1351923.49409
97 n_estimators = 300, max_depth = 30, max_features = 0.254298, Score = 1346267.90758
98 n_estimators = 300, max_depth = 13, max_features = 0.141108, Score = 1349382.44771
99 n_estimators = 300, max_depth = 19, max_features = 0.219556, Score = 1356717.57267
(100, 5)


,epoch,score,n_estimators,max_depth,max_features
95,95,1320007.78955,300,6,0.15166
60,60,1320056.71719,300,7,0.15336
22,22,1320056.71719,300,7,0.15121
33,33,1320276.54908,300,6,0.20430
26,26,1320276.54908,300,6,0.19224
78,78,1323359.97610,300,5,0.29212
32,32,1325692.96845,300,7,0.25477
89,89,1326158.12802,300,9,0.21315
18,18,1328368.41923,300,8,0.11832
30,30,1328433.34271,300,6,0.24009


In [19]:
# 가장 정확도가 높은 하이퍼패러미터를 저장
best_hyperparameters = finer_hyperparameters_list.iloc[0]

best_max_depth = best_hyperparameters["max_depth"]
best_max_features = best_hyperparameters["max_features"]

print(f"max_depth(best) = {best_max_depth}, max_features(best) = {best_max_features:.6f}")

max_depth(best) = 6.0, max_features(best) = 0.151656


In [20]:
from sklearn.ensemble import RandomForestRegressor

best_n_estimators = 3000
model = RandomForestRegressor(n_estimators=best_n_estimators,
                              max_depth=best_max_depth,
                              max_features=best_max_features,
                              random_state=37,
                              n_jobs=-1)
model

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=6.0, max_features=0.15165616098709847,
                      max_leaf_nodes=None, max_samples=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=3000,
                      n_jobs=-1, oob_score=False, random_state=37, verbose=0,
                      warm_start=False)

In [21]:
model.fit(X_train, Y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=6.0, max_features=0.15165616098709847,
                      max_leaf_nodes=None, max_samples=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=3000,
                      n_jobs=-1, oob_score=False, random_state=37, verbose=0,
                      warm_start=False)

In [22]:
prediction = model.predict(X_test)

In [23]:
prediction

array([1202964.57012691, 1255008.79707991, 1438631.66555738,
       2503246.34201609, 1490424.26219567,  237810.98195795,
        135853.69163491,  114276.23575197,  143638.88978877,
        438788.6473053 ,  446652.92366167, 1051074.14804265,
        963685.95473335,  201802.60812928, 1342275.85834235,
       2173072.66768164,  298253.94087779, 1034046.45135115,
       1006910.94113107,  619530.14901474,  220104.23965746,
        292528.22720256,  217542.28368121,  579303.4685836 ,
        117274.66039379, 2369906.65720154,  169206.89848877,
       1512772.50724245,  121958.43420081,  205906.2572791 ,
       1375341.4968778 ,  240775.55735974,  103332.11818271,
        660613.06085091,   88271.70243404,  139164.83731189,
        386136.04952716,   73430.33386115,  122523.03399621,
         94291.96981749,  129904.87214366,  373149.85856493,
       1240813.2843516 ,  393096.5565739 , 1002831.60917323,
       1370926.96360978,  229562.68640359,  129662.1798806 ,
        609716.70649963,

### 최적의 cv 찾기
* 35로 사용 : 1454429.7489887767	
* 추가적으로 더 해볼 수 있으면 ? 해봐도 될듯
* overfitting 일어나는 것 같아.........ㅠ

In [37]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100,
                              n_jobs=-1,
                              random_state=37)

X_train, X_test, Y_train = df.drop(['box_off_num'], axis=1), testdf, df['box_off_num']

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

n_estimators = 100

# 랜덤 서치 횟수
num_epoch = 50

In [38]:
cv_cv = []
for cv_turn in range(35,51, 1) : # 5, 7, 9, 11, 13, 15, 17, 19


# hyperparameter 탐색 결과가 저장될 리스트
    coarse_hyperparameters_list = []

    for epoch in range(num_epoch):
        # max depth의 값을 2~100으로 설정
        max_depth = np.random.randint(low=2, high=10)

        # 적용할 feature의 비율을 0.1~1.0로 설정
        max_features = np.random.uniform(low=0.1, high=1.0)

        model = RandomForestRegressor(n_estimators=n_estimators,
                                      max_depth=max_depth,
                                      max_features=max_features,
                                      n_jobs=-1,
                                      random_state=37)

        # cross_val_score를 실행하여 rmse의 평균 계산
        # cv는 데이터 조각 개수
        score = cross_val_score(model, X_train, Y_train, cv=cv_turn, scoring=rmse_score).mean() 

        # hyperparameter 탐색 결과를 저장할 딕셔너리
        hyperparameters = {
            'epoch': epoch,
            'score': score,
            'n_estimators': n_estimators,
            'max_depth': max_depth,
            'max_features': max_features,
            'cv' : cv_turn
        }

        # hyperparameter 탐색 결과 저장
        coarse_hyperparameters_list.append(hyperparameters)

        # hyperparameter 탐색 결과를 출력
        print(f"{epoch:2} n_estimators = {n_estimators}, max_depth = {max_depth:2}, max_features = {max_features:.6f}, Score = {score:.5f}, CV = {cv_turn}")

    # coarse_hyperparameters_list를 DataFrame으로 변환
    coarse_hyperparameters_list = pd.DataFrame.from_dict(coarse_hyperparameters_list)

    # coarse_hyperparameters_list를 score가 낮은 순으로 정렬
    coarse_hyperparameters_list = coarse_hyperparameters_list.sort_values(by="score")

    # coarse_hyperparameters_list의 상위 1개 저장
    cv_cv.append(coarse_hyperparameters_list.head(1))

 0 n_estimators = 100, max_depth =  4, max_features = 0.858669, Score = 1282875.46143, CV = 35
 1 n_estimators = 100, max_depth =  2, max_features = 0.472122, Score = 1250527.83485, CV = 35
 2 n_estimators = 100, max_depth =  4, max_features = 0.420797, Score = 1253739.93230, CV = 35
 3 n_estimators = 100, max_depth =  4, max_features = 0.113389, Score = 1275597.00407, CV = 35
 4 n_estimators = 100, max_depth =  6, max_features = 0.131492, Score = 1251926.84632, CV = 35
 5 n_estimators = 100, max_depth =  6, max_features = 0.872735, Score = 1297226.98422, CV = 35
 6 n_estimators = 100, max_depth =  7, max_features = 0.729731, Score = 1317374.45949, CV = 35
 7 n_estimators = 100, max_depth =  6, max_features = 0.551336, Score = 1290898.28891, CV = 35
 8 n_estimators = 100, max_depth =  4, max_features = 0.385187, Score = 1253739.93230, CV = 35
 9 n_estimators = 100, max_depth =  8, max_features = 0.922225, Score = 1319473.51932, CV = 35
10 n_estimators = 100, max_depth =  4, max_feature

37 n_estimators = 100, max_depth =  2, max_features = 0.668918, Score = 1236368.06212, CV = 36
38 n_estimators = 100, max_depth =  3, max_features = 0.397220, Score = 1223935.02581, CV = 36
39 n_estimators = 100, max_depth =  5, max_features = 0.868501, Score = 1264615.10049, CV = 36
40 n_estimators = 100, max_depth =  7, max_features = 0.849098, Score = 1295226.23006, CV = 36
41 n_estimators = 100, max_depth =  9, max_features = 0.549450, Score = 1275113.92742, CV = 36
42 n_estimators = 100, max_depth =  3, max_features = 0.796765, Score = 1234777.13915, CV = 36
43 n_estimators = 100, max_depth =  4, max_features = 0.719610, Score = 1230265.55383, CV = 36
44 n_estimators = 100, max_depth =  3, max_features = 0.998169, Score = 1243456.62252, CV = 36
45 n_estimators = 100, max_depth =  2, max_features = 0.761761, Score = 1234877.84026, CV = 36
46 n_estimators = 100, max_depth =  8, max_features = 0.255908, Score = 1230087.64153, CV = 36
47 n_estimators = 100, max_depth =  6, max_feature

24 n_estimators = 100, max_depth =  5, max_features = 0.302739, Score = 1251039.29638, CV = 38
25 n_estimators = 100, max_depth =  6, max_features = 0.847162, Score = 1310003.86728, CV = 38
26 n_estimators = 100, max_depth =  9, max_features = 0.694569, Score = 1315809.69574, CV = 38
27 n_estimators = 100, max_depth =  9, max_features = 0.500583, Score = 1291428.66414, CV = 38
28 n_estimators = 100, max_depth =  5, max_features = 0.251424, Score = 1234151.80175, CV = 38
29 n_estimators = 100, max_depth =  3, max_features = 0.880469, Score = 1273991.44304, CV = 38
30 n_estimators = 100, max_depth =  4, max_features = 0.744844, Score = 1264425.97659, CV = 38
31 n_estimators = 100, max_depth =  2, max_features = 0.206652, Score = 1303300.58284, CV = 38
32 n_estimators = 100, max_depth =  5, max_features = 0.185557, Score = 1238122.28584, CV = 38
33 n_estimators = 100, max_depth =  5, max_features = 0.203520, Score = 1238122.28584, CV = 38
34 n_estimators = 100, max_depth =  7, max_feature

11 n_estimators = 100, max_depth =  4, max_features = 0.301953, Score = 1220005.03971, CV = 40
12 n_estimators = 100, max_depth =  7, max_features = 0.650254, Score = 1275712.50346, CV = 40
13 n_estimators = 100, max_depth =  4, max_features = 0.471109, Score = 1226283.53597, CV = 40
14 n_estimators = 100, max_depth =  3, max_features = 0.181642, Score = 1238255.23399, CV = 40
15 n_estimators = 100, max_depth =  2, max_features = 0.401451, Score = 1211785.16377, CV = 40
16 n_estimators = 100, max_depth =  5, max_features = 0.169624, Score = 1213291.45488, CV = 40
17 n_estimators = 100, max_depth =  8, max_features = 0.204127, Score = 1230278.90052, CV = 40
18 n_estimators = 100, max_depth =  7, max_features = 0.657787, Score = 1275712.50346, CV = 40
19 n_estimators = 100, max_depth =  8, max_features = 0.557649, Score = 1285364.90228, CV = 40
20 n_estimators = 100, max_depth =  4, max_features = 0.184920, Score = 1215757.57694, CV = 40
21 n_estimators = 100, max_depth =  6, max_feature

KeyboardInterrupt: 

In [39]:
display(cv_cv)

[    epoch         score  n_estimators  max_depth  max_features  cv
 14     14 1240169.81514           100          4       0.27318  35,
    epoch         score  n_estimators  max_depth  max_features  cv
 2      2 1210819.66532           100          9       0.17655  36,
    epoch         score  n_estimators  max_depth  max_features  cv
 1      1 1203055.19452           100          7       0.12941  37,
    epoch         score  n_estimators  max_depth  max_features  cv
 7      7 1229001.23809           100          9       0.20166  38,
     epoch         score  n_estimators  max_depth  max_features  cv
 24     24 1191898.82258           100          8       0.13186  39]

In [14]:
display(cv_cv)

[    epoch         score  n_estimators  max_depth  max_features  cv
 60     60  1.267566e+06           100          5      0.221966  27,
     epoch         score  n_estimators  max_depth  max_features  cv
 54     54  1.246747e+06           100          7        0.1176  29,
     epoch         score  n_estimators  max_depth  max_features  cv
 17     17  1.259830e+06           100          9      0.142936  31,
     epoch         score  n_estimators  max_depth  max_features  cv
 88     88  1.235076e+06           100          6      0.233692  33,
     epoch         score  n_estimators  max_depth  max_features  cv
 67     67  1.229384e+06           100          6      0.170716  35]

In [35]:
display(cv_cv)

[    epoch         score  n_estimators  max_depth  max_features  cv
 25     25  1.303383e+06           100          5      0.234854  19,
     epoch         score  n_estimators  max_depth  max_features  cv
 56     56  1.277253e+06           100          8      0.174468  21,
     epoch         score  n_estimators  max_depth  max_features  cv
 12     12  1.328894e+06           100          7      0.178809  23,
     epoch         score  n_estimators  max_depth  max_features  cv
 60     60  1.266826e+06           100          9      0.110251  25]

In [ ]:
# 가장 정확도가 높은 하이퍼패러미터를 저장
best_hyperparameters = finer_hyperparameters_list.iloc[0]

best_max_depth = best_hyperparameters["max_depth"]
best_max_features = best_hyperparameters["max_features"]

print(f"max_depth(best) = {best_max_depth}, max_features(best) = {best_max_features:.6f}")

In [ ]:
from sklearn.ensemble import RandomForestRegressor

best_n_estimators = 3000
model = RandomForestRegressor(n_estimators=best_n_estimators,
                              max_depth=best_max_depth,
                              max_features=best_max_features,
                              random_state=37,
                              n_jobs=-1)
model

In [ ]:
model.fit(X_train, Y_train)
prediction = model.predict(X_test)

## tensorflow

In [16]:
train_Y = df['box_off_num']
train_X = df.drop(['box_off_num'], axis=1)

test_X = testdf

In [24]:
import tensorflow as tf
import numpy as np

X = tf.placeholder(shape = [None,11], dtype=tf.float32)
Y = tf.placeholder(shape = [None,1], dtype=tf.float32)

W = tf.Variable(tf.random_normal([11,1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name = 'bias')

H = tf.matmul(X, W) + b
cost = tf.reduce_mean(tf.square(H-Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.0000001)
train = optimizer.minimize(cost)


sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(5000) :
    _, cost_val = sess.run([train, cost],
                          feed_dict = {
                              X : train_X.values,
                              Y : train_Y.values.reshape(-1,1)
                          })
    if step % 500 == 0 :
        print('cost값은 : {}'.format(cost_val))
    
prediction = sess.run(H, feed_dict = {X : test_X.values, })

cost값은 : 3833388597248.0
cost값은 : 2351955443712.0
cost값은 : 2347802558464.0
cost값은 : 2347764809728.0
cost값은 : 2347744100352.0
cost값은 : 2347723653120.0
cost값은 : 2347703730176.0
cost값은 : 2347683807232.0
cost값은 : 2347663622144.0
cost값은 : 2347643174912.0


# 저장

In [70]:
# 저장함수
def save_file(prediction, number) :
    sub = pd.read_csv('./data/submission.csv')
    sub['box_off_num'] = prediction
    sub.to_csv('./submission/submission' + str(number) + '.csv', index=False)
    return print('done')

In [71]:
save_file(prediction, 19)

done


In [160]:
a = 0
for i in range(len(prediction)) :
    if prediction[i] <= 0 :
        prediction[i] = 0
        a += 1

print(a)
#prediction = prediction.astype(int)
prediction

71


array([1115566.30565657, 1330898.07271574, 1956710.48447819,
       2166859.56483689, 2080734.17510612,  346045.45931293,
             0.        ,       0.        ,  112417.19426519,
        951379.95868599, 1162406.66919744, 1679870.66858181,
       1124792.64272501,   49453.00864089, 1559751.63842055,
       2517328.07425241,       0.        , 1568512.0582598 ,
       1354633.42175171,  243400.04957435,       0.        ,
        450034.0288779 ,  454500.39016113,       0.        ,
             0.        , 2425606.73876284,       0.        ,
       1806573.56296934,   87379.25725834,  204802.27780387,
       1488231.30625899,  218700.55519663,       0.        ,
       1374517.85637328,   86868.77794634,       0.        ,
             0.        ,       0.        ,   22318.97752276,
             0.        ,  204638.03387941,  397508.22867611,
        374282.20315428,  543604.79166777,  867479.27053258,
       2054273.09964444,       0.        ,  273394.65591729,
        314385.90759814,